# Generative Question Answering






## Installation

1. Click File -> Save a Copy in Drive. This will create a copy for you to modify in your Google Drive.

2. This project will require you to install langchain, cohere, chromadb, and other libraries. Run the command below to start the installation. Installation may take a couple of minutes.


In [ ]:
!pip install cohere==4.57 nltk==3.8 unstructured langchain==0.0.195 chromadb==0.3.26 > /dev/null 2>&1

## Import Libraries




In [ ]:
import os
import nltk
nltk.download('punkt')

from langchain import Cohere
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# fixes nltk path issue on colab
!cp -R ~/nltk_data/tokenizers/punkt/PY3 ~/nltk_data/tokenizers/punkt/PY3_tab

## Setup

Before running the next cell, make sure you have a [Cohere API key](https://dashboard.cohere.ai/api-keys). Enter your API key using getpass in the cell below.


In [ ]:
from getpass import getpass
api_key = getpass()
os.environ["COHERE_API_KEY"] = api_key

··········


## Uploading documents to Colab

There are two ways you can upload your documents to Colab.

#### Method 1

1. Right-click on the link below and save the text file to your computer.

[Wikipedia Article: Quantum Computing](https://raw.githubusercontent.com/Thinkful-Ed/ai-in-web-dev-resources/refs/heads/main/books/quantumcomputers.txt)

2. Click on the folder icon tab at the left of the Colab editor. Create a new *documents* directory. Click on the *upload file* icon and select the file that you want to upload.

#### Method 2

Run the command below:

In [ ]:
!curl -o /content/documents/quantumcomputers.txt https://raw.githubusercontent.com/Thinkful-Ed/ai-in-web-dev-resources/refs/heads/main/books/quantumcomputers.txt --create-dirs

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  115k  100  115k    0     0   449k      0 --:--:-- --:--:-- --:--:--  449k


## Load the documents

The [DirectoryLoader](https://js.langchain.com/docs/api/document_loaders_fs_directory/classes/DirectoryLoader) class will allow you to load multiple documents that are in a directory. Create a directory by clicking in the folder icon at the left and upload your text documents there. If you want to use another format, such as `md` files, make sure to change the `glob="**/*.txt"` variable and read the documentation to make sure that the file type is supported.

**Note:** Cohere is now imposing a limit on embeddings. If you get an error try a smaller text (like a wikipedia article).

In [ ]:
loader = DirectoryLoader('/content/documents', glob="**/*.txt") # import all txt files in the documents folder
documents = loader.load()
print (f'You have {len(documents)} document(s) in your documents folder')

You have 1 document(s) in your documents folder


## Splitting Text

Splitting text is useful because when you have a long document, it can be difficult to find the specific information relevant to a question. The `CharacterTextSplitter` helps break down the text into smaller, more manageable chunks.

In [ ]:
#text_splitter = CharacterTextSplitter(chunk_size=1500)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)
print (f'Your document(s) was/were splitted into {len(texts)} chunks.')

Your document(s) was/were splitted into 175 chunks.


## Embeddings and RetrievalQA

In simple terms, this code sets up the necessary tools for question-answering. It converts text into numerical representations called embeddings, creates a search index for the documents, and sets up a retrieval-based question-answering system using a specific type of chain (RetrievalQA).


In [ ]:
embeddings = CohereEmbeddings(cohere_api_key=os.getenv("COHERE_API_KEY"))
docsearch = Chroma.from_documents(texts, embeddings, persist_directory = 'db')
docsearch.persist() # save db
qa = RetrievalQA.from_chain_type(llm=Cohere(), retriever=docsearch.as_retriever(), return_source_documents=True)

In [ ]:
query = "what is quantum computing?"
result = qa({"query": query})
result['result']

' Quantum computing is a method of computing that leverages quantum mechanical phenomena. On small scales, physical matter exhibits properties of both particles and waves, and quantum computing leverages this behavior using specialized hardware. Classical physics cannot explain the operation of these quantum devices, and a scalable quantum computer could perform some calculations exponentially faster than any modern "classical" computer. In particular, a large-scale quantum computer could break widely used encryption schemes and aid physicists in performing physical simulations. \n\nThe basic unit of information in quantum computing is the qubit, which serves a similar function to the bit in classical computing. However, unlike a classical bit, which can be in one of two states (binary), a qubit can exist in a quantum superposition of its two basis states, meaning it is in both states simultaneously. When measuring a qubit, the result is a probabilistic output of a classical bit. If a 

You can also get the sources:

In [ ]:
result['source_documents']

[Document(page_content='A \'\'\'quantum computer\'\'\' is a [[computer]] that exploits [[quantum mechanical]] phenomena. On small scales, physical matter exhibits properties of [[wave-particle duality|both particles and waves]], and quantum computing leverages this behavior using specialized hardware. [[Classical physics]] cannot explain the operation of these quantum devices, and a scalable quantum computer could perform some calculations [[Exponential growth|exponentially]] faster{{efn|As used in this article, "exponentially faster" has a precise [[computational complexity|complexity theoretical]] meaning. Usually, it means that as a function of input size in bits, the best known classical algorithm for a problem requires an [[exponential growth|exponentially growing]] number of steps, while a quantum algorithm uses only a polynomial number of steps.}} than any modern "classical" computer. In particular, a large-scale quantum computer could [[post-quantum cryptography|break widely us

In [ ]:
sources = ""
for count, source in enumerate(result['source_documents'],1):
  sources += "Source " + str(count) + "\n"
  sources += source.page_content + "\n"

print(sources)


Source 1
A '''quantum computer''' is a [[computer]] that exploits [[quantum mechanical]] phenomena. On small scales, physical matter exhibits properties of [[wave-particle duality|both particles and waves]], and quantum computing leverages this behavior using specialized hardware. [[Classical physics]] cannot explain the operation of these quantum devices, and a scalable quantum computer could perform some calculations [[Exponential growth|exponentially]] faster{{efn|As used in this article, "exponentially faster" has a precise [[computational complexity|complexity theoretical]] meaning. Usually, it means that as a function of input size in bits, the best known classical algorithm for a problem requires an [[exponential growth|exponentially growing]] number of steps, while a quantum algorithm uses only a polynomial number of steps.}} than any modern "classical" computer. In particular, a large-scale quantum computer could [[post-quantum cryptography|break widely used encryption schemes

## Download the database

Zip the database into `db.zip` and download it to your computer.


In [ ]:
!zip -r /content/db.zip /content/db

In [ ]:
from google.colab import files
files.download("/content/db.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>